In [ ]:
# loading packages
# basic + dates 
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn import metrics
# data visualization
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
# %matplotlib inline
import statsmodels.api as sm
# time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from timeit import default_timer as timer
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

#### T1

In [ ]:
def load_the_dataset(dataset_location):
    
    try:
        # code starts here
        Store50 = pd.read_excel(dataset_location,parse_dates = True, index_col = 'Date')
        # code ends here
        return Store50
    except :
        return "File not found. Please check the file"


In [ ]:
# store the result of the dataset
dataset_location = "store50.xlsx"
Store50=load_the_dataset(dataset_location)
Store50.head()

#### T2

In [ ]:
## List the number of unique stores for which we have the data?
def unique_store(Store50):
    uniq = None
    # code starts here
    uniq=Store50['Store'].unique()
    # code ends here
    return len(uniq)


In [ ]:
unique_store(Store50)

#### T3

In [ ]:
def no_obs_store(Store50):
    store_counts = {}
    # code starts here
    uniq=Store50['Store'].unique()
    for i in uniq:
        count = len(Store50[Store50['Store'] == i])
        store_counts[i] = count
        # print(f'Total number of observations in {i} = {count}')
    #code ends here
    return store_counts

In [ ]:
no_obs_store(Store50)

#### T4

In [ ]:
def less_obs_stores(Store50):
    less_obs_stores_list = []
    # code Starts here
    uniq = Store50['Store'].unique()
    for i in uniq:
        if len(Store50[Store50['Store'] == i]) < 852:
            less_obs_stores_list.append(i)
    # code ends here
    return less_obs_stores_list


In [ ]:
less_obs_stores(Store50)

#### T5

In [ ]:
# Here, we are also reversing the data as we need to contiguous data in the ascending order for a Time Series Analysis.
def avg_cust_store(Store50):
    mean_1041 , mean_1044 = None, None
    # code starts here
    Store50_1044 = Store50[Store50.Store == 1044].sort_index(ascending=True)
    Store50_1041 = Store50[Store50.Store == 1041].sort_index(ascending=True)
    mean_1041 = round(Store50_1041.Customers.mean(),0)
    mean_1044 = round(Store50_1044.Customers.mean(),0)
    # code ends here
    return mean_1044 , mean_1041


In [ ]:
avg_cust_store(Store50)

#### T6

In [ ]:
def time_series_plot(Store50):
    df = None
    # code starts here
    Store50_1044 = Store50[Store50.Store == 1044]. sort_index(ascending=True)
    Store50_1041 = Store50[Store50.Store == 1041]. sort_index(ascending=True)
    df = pd.DataFrame(columns=['Store_1041','Store_1044'])
    df['Store_1044'] = Store50_1044['Customers']
    df['Store_1041'] = Store50_1041['Customers']
    # code ends here
    return df


In [ ]:
df = time_series_plot(Store50)
print(df)

#### T8

In [ ]:
def store_id_max_weekly(df):
    max_id = None
    # code starts here
    max_id = df.resample('W').sum().sum().idxmax()
    # code ends here
    return max_id

In [ ]:
store_id_max_weekly(df)

In [ ]:
def store_id_max_monthly(df):
    max_id =  None
    # code starts here
    max_id = df.resample('M').sum().sum().idxmax()
    # code ends here
    return max_id


In [ ]:
store_id_max_monthly(df)

#### T1.9

In [ ]:
def weekly_1041(df):
    df_weekly_1041 =  None
    #code starts here
    df_weekly_1041= df.resample('W').sum()
    df_weekly_1041.drop('Store_1044', axis=1, inplace=True)
    # code ends here
    return df_weekly_1041

In [ ]:
df_weekly_1041 = weekly_1041(df)

In [ ]:
def weekly_1044(df):
    df_weekly_1044 = None
    #code starts here
    df_weekly_1044= df.resample('W').sum()
    df_weekly_1044.drop('Store_1041', axis=1, inplace=True)
    # code ends here
    return df_weekly_1044

In [ ]:
df_weekly_1044 = weekly_1044(df)

#### T2.1

In [ ]:
## Defning a function
def adf_test(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(window=7).mean()
    rolstd = timeseries.rolling(window=7).std()

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    dftest = adfuller(timeseries,regression='ct')#running the adf test on the input time series
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    ## creating a series to format the output
    for key,value in dftest[4].items():##running a for loop to format the critical values of the test statistic
       dfoutput['Critical Value (%s)'%key] = value
    if dfoutput[1] <= 0.05:
        return "Data is stationary"
    else:
        return "Data is non-stationary"

#### T2.2

In [ ]:
def train_test_split_1041(df_weekly_1041):# Split the data into train and test ( consider year 2016 for test data) 
    train_1041=df_weekly_1041[df_weekly_1041.index.year !=2016]
    test_1041=df_weekly_1041[df_weekly_1041.index.year ==2016]
    return train_1041,test_1041

In [ ]:
train_1041=train_test_split_1041(df_weekly_1041)[0]
test_1041=train_test_split_1041(df_weekly_1041)[1]

In [ ]:
def train_test_split_1044(df_weekly_1044):# Split the data into train and test ( consider year 2016 for test data) 
    train_1044=df_weekly_1044[df_weekly_1044.index.year !=2016]
    test_1044=df_weekly_1044[df_weekly_1044.index.year ==2016]
    return train_1044,test_1044


In [ ]:
train_1044=train_test_split_1044(df_weekly_1044)[0]
test_1044=train_test_split_1044(df_weekly_1044)[1]